In [1]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
import json
import numpy as np
from transformers import BitsAndBytesConfig  #

/home/mgalindo/max/projects/tlama-commerce/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('eigencore_dataset.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [4]:
train_data = []
for example in data:
    if 'input' in example and ('target' in example or 'output' in example):
        question = example['input']
        answer = example['target'] if 'target' in example else example['output']
        train_data.append({
            "input_text": question,
            "target_text": answer
        })

In [5]:
train_data[0:3]  # Display the first 3 examples

[{'input_text': '¿Qué es EigenCore?',
  'target_text': 'EigenCore es una startup de inteligencia artificial nacida en México. Fuimos fundados por dos estudiantes de maestría de la UNAM con la misión de hacer la IA más accesible, segura, eficiente y poderosa. Nuestro objetivo es democratizar el acceso a tecnologías avanzadas de IA, especialmente en Latinoamérica.'},
 {'input_text': 'Cuéntame sobre EigenCore, ¿qué hacen?',
  'target_text': 'EigenCore es una startup de inteligencia artificial nacida en México. Investigamos nuevas formas de crear IA más eficiente y desarrollamos productos que ayudan a personas y negocios en su día a día. Nos especializamos en crear soluciones de IA adaptadas específicamente a las necesidades de nuestros clientes, haciendo la tecnología más accesible y útil.'},
 {'input_text': '¿Qué hacen en EigenCore?',
  'target_text': 'En EigenCore nos especializamos en investigación y desarrollo de soluciones de inteligencia artificial. Trabajamos con diversas tecnologí

In [12]:
# Convertir a formato de Hugging Face datasets
train_dataset = Dataset.from_list(train_data)

In [6]:
model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [7]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Cargar el modelo con la configuración de cuantización
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
)

In [8]:
# Preparar el modelo para entrenamiento con QLoRA
model = prepare_model_for_kbit_training(model)

# Configurar LoRA para T5
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
    # Para T5, los módulos objetivo típicamente incluyen:
    target_modules=["q", "v", "k", "o", "wi", "wo"]  
)

In [9]:
model = get_peft_model(model, lora_config)

In [10]:
# Tokenizar el dataset
def tokenize_function(examples):
    # T5 espera que la entrada tenga un prefijo que indique la tarea
    inputs = ["responde: " + text for text in examples["input_text"]]
    
    model_inputs = tokenizer(
        inputs, 
        max_length=512, 
        padding="max_length", 
        truncation=True
    )
    
    labels = tokenizer(
        examples["target_text"], 
        max_length=512, 
        padding="max_length", 
        truncation=True
    )
    
    model_inputs["labels"] = labels["input_ids"]
    
    # Reemplazar padding token id con -100 en labels para que no se consideren en la pérdida
    for i in range(len(model_inputs["labels"])):
        model_inputs["labels"][i] = [
            -100 if token == tokenizer.pad_token_id else token 
            for token in model_inputs["labels"][i]
        ]
    
    return model_inputs

In [13]:
tokenized_dataset = train_dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 309/309 [00:00<00:00, 587.35 examples/s]


In [14]:
sample = tokenized_dataset[0]
print("Input IDs:", sample["input_ids"][:20])  # Primeros 20 tokens
print("Labels:", [l for l in sample["labels"][:20] if l != -100])  # Labels no ignorados

Input IDs: [3531, 15, 10, 3, 2, 5991, 154, 3, 15, 7, 17213, 13026, 15, 58, 1, 0, 0, 0, 0, 0]
Labels: [17213, 13026, 15, 3, 15, 7, 73, 9, 11459, 20, 16, 1625, 2211, 4915, 7353, 3, 29, 9, 10812, 9]


In [15]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-large-eigencore",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=1e-3,
    num_train_epochs=3,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    warmup_steps=100,
    # evaluation_strategy="no",  # Eliminado porque parece causar problemas
    save_total_limit=3,
    predict_with_generate=True
    use_cache=False
)

# Configurar el entrenador
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Iniciar entrenamiento
trainer.train()

# Guardar el modelo adaptado con LoRA
model.save_pretrained("./flan-t5-large-eigencore-lora")
tokenizer.save_pretrained("./flan-t5-large-eigencore-lora")

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/home/mgalindo/max/projects/tlama-commerce/env/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss


KeyboardInterrupt: 